Search_results.json Is everything that we need at the time of running the script

EveryResults_Backup.json Is everything so we never have to rerun the script for the same author ever again.

In [ ]:
from serpapi import GoogleSearch
import json
import os
import time
import KeyToSerpapi

def get_all_results(query, max_pages=20):
    all_results = []
    start = 0
    page = 1
    everything = []

    while page <= max_pages:
        params = {
            "engine": "google_scholar",
            "q": query,
            "api_key": KeyToSerpapi.api_key,
            "start": start,
            "num": 20  # Request maximum number of results per page
        }
        try:
            search = GoogleSearch(params)
            results = search.get_dict()
            organic_results = results.get("organic_results", [])
            
            if not organic_results:
                break  # No more results
            
            for result in organic_results:
                publication_info = result.get("publication_info", {})
                authors = publication_info.get("authors", [])
                result_data = {
                    "resources": result.get("resources", []),
                    "title": result.get("title"),
                    "authors": authors,  # Extract authors from publication_info
                    "snippet": result.get("snippet"), # Description
                }
                all_results.append(result_data)
                everything.append(result)
            
            print(f"Retrieved {len(organic_results)} results from page {page}")
            start += 20
            page += 1
            time.sleep(2)  # Add a delay to avoid hitting rate limits
        except Exception as e:
            print(f"An error occurred: {e}")
            break

    return all_results, everything

def file_authors(file_path):
    with open(file_path, 'r') as file:
        authors = [line.strip() for line in file]
    return authors

def append_to_json_file(file_path, new_data):
    if os.path.exists(file_path):
        # Read existing data
        with open(file_path, 'r') as file:
            try:
                existing_data = json.load(file)
                if not isinstance(existing_data, dict):
                    existing_data = {}  # Ensure existing_data is a dictionary
            except json.JSONDecodeError:
                existing_data = {}  # If file is empty or corrupt
    else:
        existing_data = {}
    
    # Update existing data with new data
    existing_data.update(new_data)
    
    # Write combined data back to file
    with open(file_path, 'w') as file:
        json.dump(existing_data, file, indent=2)

query = "author:"
file = 'authors.txt' # File with the authors
authors = file_authors(file)
allFilteredResults = {}
allResults = {}

for author in authors:
    print(f"Searching for: {author}")
    results, every_result = get_all_results(query + author)
    
    allFilteredResults[author] = results
    allResults[author] = every_result

# Define the file paths
results_file_path = 'Search_results.json'
every_results_file_path = 'EveryResults_Backup.json'

try:
    # Append results to the JSON files
    append_to_json_file(results_file_path, allFilteredResults)
    append_to_json_file(every_results_file_path, allResults)

    # Calculate the total number of results
    total_results = sum(len(results) for results in allFilteredResults.values())
    print(f"Total results retrieved: {total_results}")
    print(f"Results have been written to {results_file_path}")
except IOError as e:
    print(f"An error occurred while writing files: {e}")
